<a href="https://colab.research.google.com/github/viswambhar-yasa/Where_r_Bees/blob/main/AMGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.utils.data
import torchvision.datasets as datasets
import torchvision.transforms as transforms

def weight_initialization(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)


class DCGAN(nn.Module):
    def __init__(self, z_size, output_size, n_classes, n_output_channels=1):
        super(DCGAN, self).__init__()
        self.output_size=output_size
        self.n_classes=n_classes
        self.Gen = nn.Sequential(
            nn.ConvTranspose2d(in_channels=z_size, out_channels=output_size*16,
                               kernel_size=4, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(output_size*16),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(in_channels=output_size*16, out_channels=output_size*8,
                               kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(output_size*8),
            nn.ReLU(True),

            nn.ConvTranspose2d(in_channels=output_size*8, out_channels=output_size*4,
                               kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(output_size*4),
            nn.ReLU(True),

            nn.ConvTranspose2d(in_channels=output_size*4, out_channels=output_size*2,
                               kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(in_channels=output_size*2, out_channels=output_size,
                               kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(output_size),
            nn.ReLU(True),

            nn.ConvTranspose2d(in_channels=output_size, out_channels=n_output_channels,
                               kernel_size=4, stride=2, padding=1, bias=False),
            nn.Tanh()
            )
        
        self.Dis = nn.Sequential(
            nn.Conv2d(in_channels=n_output_channels, out_channels=output_size,
                      kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(in_channels=output_size, out_channels=output_size*2,
                      kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(output_size*2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(in_channels=output_size*2, out_channels=output_size*4,
                      kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(output_size*4),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(in_channels=output_size*4, out_channels=output_size*8,
                      kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(output_size*8),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(in_channels=output_size*8, out_channels=output_size*16,
                      kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(output_size*16),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(in_channels=output_size*16, out_channels=1,
                        kernel_size=4, stride=1, padding=1, bias=False),
            nn.Sigmoid())

    def forward(self):
      return [self.Gen(),self.Dis()]


In [ ]:

print('Pytorch Version', torch.__version__)
print('Is GPU available:',torch.cuda.is_available())
if torch.cuda.is_available():
    print('Number of GPUs',torch.cuda.device_count())

ngpu = torch.cuda.device_count()
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

z_vector_size=128
output_image_size=64
n_classes= 10 ## number of labels
DCGAN_Net = DCGAN(z_vector_size, output_image_size,
                  n_classes).to(device)

if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(DCGAN_Net.Gen, list(range(ngpu)))
    netD = nn.DataParallel(DCGAN_Net.Dis, list(range(ngpu)))
else:
    netG = DCGAN_Net.Gen
    netD = DCGAN_Net.Dis
netG.apply(weight_initialization)
netD.apply(weight_initialization)



Pytorch Version 1.12.0+cu113
Is GPU available: True
Number of GPUs 1


Sequential(
  (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (1): LeakyReLU(negative_slope=0.2, inplace=True)
  (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): LeakyReLU(negative_slope=0.2, inplace=True)
  (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): LeakyReLU(negative_slope=0.2, inplace=True)
  (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): LeakyReLU(negative_slope=0.2, inplace=True)
  (11): Conv2d(512, 1024, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (12): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tr

In [ ]:
from torchvision import models
from torchsummary import summary
summary(netG,(128,1,1))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
   ConvTranspose2d-1           [-1, 1024, 2, 2]       2,097,152
       BatchNorm2d-2           [-1, 1024, 2, 2]           2,048
              ReLU-3           [-1, 1024, 2, 2]               0
   ConvTranspose2d-4            [-1, 512, 4, 4]       8,388,608
       BatchNorm2d-5            [-1, 512, 4, 4]           1,024
              ReLU-6            [-1, 512, 4, 4]               0
   ConvTranspose2d-7            [-1, 256, 8, 8]       2,097,152
       BatchNorm2d-8            [-1, 256, 8, 8]             512
              ReLU-9            [-1, 256, 8, 8]               0
  ConvTranspose2d-10          [-1, 128, 16, 16]         524,288
      BatchNorm2d-11          [-1, 128, 16, 16]             256
             ReLU-12          [-1, 128, 16, 16]               0
  ConvTranspose2d-13           [-1, 64, 32, 32]         131,072
      BatchNorm2d-14           [-1, 64,

In [ ]:
summary(netD,(1,64,64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,024
         LeakyReLU-2           [-1, 64, 32, 32]               0
            Conv2d-3          [-1, 128, 16, 16]         131,072
       BatchNorm2d-4          [-1, 128, 16, 16]             256
         LeakyReLU-5          [-1, 128, 16, 16]               0
            Conv2d-6            [-1, 256, 8, 8]         524,288
       BatchNorm2d-7            [-1, 256, 8, 8]             512
         LeakyReLU-8            [-1, 256, 8, 8]               0
            Conv2d-9            [-1, 512, 4, 4]       2,097,152
      BatchNorm2d-10            [-1, 512, 4, 4]           1,024
        LeakyReLU-11            [-1, 512, 4, 4]               0
           Conv2d-12           [-1, 1024, 2, 2]       8,388,608
      BatchNorm2d-13           [-1, 1024, 2, 2]           2,048
        LeakyReLU-14           [-1, 102

In [ ]:
criterion = nn.BCELoss()
criterion2=nn.CrossEntropyLoss()
# Learning rate for optimizers
lr = 0.0001

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [ ]:
workers = 2

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.Resize(64),
        transforms.CenterCrop(64),
        transforms.ToTensor()
    ])
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.Resize(64),
        transforms.CenterCrop(64),
        transforms.ToTensor(),
    ])
)


# Create the dataloader
dataloader = torch.utils.data.DataLoader(training_data, batch_size=32,
                                         shuffle=True, num_workers=workers)

In [ ]:
num_epochs=50
max_steps=5

In [ ]:
dloss=[]
gloss=[]
for epoch in range(num_epochs):
  for steps,data in enumerate(dataloader,0):
    if steps<max_steps:

      # discrimination training
      optimizerD.zero_grad()
      input_data,label=data[0].to(device),data[1]
      batch_size=input_data.size(0)
      real_label = torch.full((batch_size,), 1., dtype=torch.float, device=device)
      
      # training on real data 
      real_output = netD(input_data).squeeze()
      # loss from real labels   
      errD_real = criterion(real_output, real_label)
      errD_real.backward(retain_graph=True)
      D_x0 = real_output.mean().item()

      # input latent vectors to generate fake image 
      noise = torch.randn(batch_size,z_vector_size , 1, 1, device=device)
        # Generate fake image batch with Generator
      fake_label=torch.full((batch_size,), 0., dtype=torch.float, device=device)
      gen_data=netG(noise)
      # discrimation training on fake images
      fake_output = netD(gen_data).squeeze()
      # loss for fake image
      errD_fake= criterion(fake_output, fake_label)
      errD_fake.backward(retain_graph=True)
      D_x1=fake_output.mean().item()
      total_Dloss=errD_real+errD_fake
      optimizerD.step()

      optimizerG.zero_grad()
      fake_output_gen = netD(gen_data).squeeze()
      errG_label= criterion(fake_output_gen, real_label)
      # Calculate gradients for G
      errG_label.backward(retain_graph=True)
      D_G_z2 = fake_output_gen.mean().item()
      # Update G
      optimizerG.step()
    else:
      break
  dloss.append(total_Dloss.item())
  gloss.append(errG_label.item())
  print('Epoch: ',epoch,'   Discrimator Loss: ',total_Dloss.item() ,'  Generator Loss: ',errG_label.item())

Epoch:  0    Discrimator Loss:  0.0745401680469513   Generator Loss:  4.5229878425598145
Epoch:  1    Discrimator Loss:  0.07563047111034393   Generator Loss:  5.866857528686523
Epoch:  2    Discrimator Loss:  0.05290733650326729   Generator Loss:  4.963717460632324
Epoch:  3    Discrimator Loss:  0.07909831404685974   Generator Loss:  4.41572904586792
Epoch:  4    Discrimator Loss:  0.07997547090053558   Generator Loss:  4.279857635498047
Epoch:  5    Discrimator Loss:  0.09566819667816162   Generator Loss:  5.567928314208984
Epoch:  6    Discrimator Loss:  0.17214429378509521   Generator Loss:  6.1454267501831055
Epoch:  7    Discrimator Loss:  0.0809081643819809   Generator Loss:  5.508798599243164
Epoch:  8    Discrimator Loss:  0.11163375526666641   Generator Loss:  5.822798728942871
Epoch:  9    Discrimator Loss:  0.06991199404001236   Generator Loss:  5.042856216430664
Epoch:  10    Discrimator Loss:  0.050492268055677414   Generator Loss:  4.861244201660156
Epoch:  11    Discri